In [ ]:
import math
from six.moves import xrange


class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                # 타겟 값이 정상 유저인데, 정상이라고 판단한 경우 ( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 1.25*(p)*(targets[index] - p)
                der2 = 1.25*(p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 50% 더 페널티
                der1 = 1.5*(1-p)*(targets[index] - p)
                der2 = 1.5*(1-p)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [ ]:
%%time

from catboost import CatBoostClassifier, Pool


for i in range(n_splits):
    print('='*20, str(i+1), '='*20) 
    
#     cat_features = list(range(len(list(past_6m_data[cat_cols].columns))))
    cat_feat_ls = []
    for item in list(data[cat_cols].select_dtypes(include='category').columns):
        cat_feat_ls.append(item)
    
    cat_features = cat_feat_ls

    train_data = globals()[f'train_feat_{i+1}']
    train_labels = globals()[f'train_label_{i+1}']
    
    valid_data = globals()[f'valid_feat_{i+1}']
    valid_labels = globals()[f'valid_label_{i+1}']


    globals()[f'CatB_model_{i+1}'] = CatBoostClassifier(iterations=15000,
                                                        depth=16,
                                                        thread_count=12,
                                                        learning_rate=1e-3,
                                                        loss_function=LoglossObjective(),
                                                        eval_metric='PRAUC',
                                                        early_stopping_rounds=400,
                                                        
                                                        verbose=1000)

    # train the model
    globals()[f'CatB_model_{i+1}'].fit(train_data, train_labels, cat_features, eval_set = (valid_data,valid_labels))

    globals()[f'CatB_model_{i+1}'].save_model(f'CatB_C_Penalty_v6_fold_{i+1}')  
    print(str(i+1),'fold model saved')